<a href="https://colab.research.google.com/github/Sardar-moazzam10/PIAIC_Assignment/blob/main/langchain_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [248]:
!pip install langchain_google_genai

In [249]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [252]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm =ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=GOOGLE_API_KEY, temperature=0.7)

In [253]:
AIMessage=llm.invoke("What is the capital of France?")
AIMessage

AIMessage(content='Paris', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-17396714-4b20-49a1-9b66-aa78fdf80bbc-0', usage_metadata={'input_tokens': 8, 'output_tokens': 2, 'total_tokens': 10, 'input_token_details': {'cache_read': 0}})

In [254]:
from re import I
from langchain.tools import tool
# Define the tool using a decorator
from langchain.tools import tool
import datetime
import random
@tool
def get_current_date() -> str:
    """Returns the current date in YYYY-MM-DD format."""
    return datetime.date.today().strftime("%Y-%m-%d")

@tool
def get_random_number(min_val: int, max_val: int) -> int:
    """Returns a random integer between min_val and max_val (inclusive)."""
    return random.randint(min_val, max_val)

@tool
def get_current_time() -> str:
    """Returns the current time in HH:MM:SS format."""
    return datetime.datetime.now().strftime("%H:%M:%S")
@tool
def reverse_string(input_string: str) -> str:
    """Reverses a given string."""
    return input_string[::-1]
@tool
def calculate_area_of_rectangle(length: float, width: float) -> float:
    """
    Calculates the area of a rectangle.

    Args:
        length: The length of the rectangle.
        width: The width of the rectangle.

    Returns:
        The area of the rectangle.

    Raises:
        ValueError: If either length or width is negative.
    """
    if length < 0 or width < 0:
        raise ValueError("Length and width must be non-negative.")
    return length * width

tools = [get_current_date,get_random_number,get_current_time,reverse_string,calculate_area_of_rectangle]


In [255]:
llm_with_tools = llm.bind_tools(tools)

In [256]:
from langchain_core.messages import HumanMessage,AIMessage

In [257]:
query="Give me the random number between 1 and 40 "
human_message=[HumanMessage(content=query)]
print(human_message)


[HumanMessage(content='Give me the random number between 1 and 40 ', additional_kwargs={}, response_metadata={})]


In [258]:
AIMessage=llm_with_tools.invoke(human_message)
AIMessage

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_random_number', 'arguments': '{"min_val": 1.0, "max_val": 40.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-1cdb470d-edb6-4e76-9f9a-7a10e0eb1590-0', tool_calls=[{'name': 'get_random_number', 'args': {'min_val': 1.0, 'max_val': 40.0}, 'id': '2e42daa4-f549-42ad-84e7-9e7d66beef25', 'type': 'tool_call'}], usage_metadata={'input_tokens': 298, 'output_tokens': 11, 'total_tokens': 309, 'input_token_details': {'cache_read': 0}})

In [259]:
AIMessage=llm_with_tools.invoke(human_message)
human_message.append(AIMessage)
print(human_message)

[HumanMessage(content='Give me the random number between 1 and 40 ', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_random_number', 'arguments': '{"min_val": 1.0, "max_val": 40.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f7648f23-24d8-46ce-a67b-168a5e1ab651-0', tool_calls=[{'name': 'get_random_number', 'args': {'min_val': 1.0, 'max_val': 40.0}, 'id': '541faac0-9b92-4be4-a7f3-b7dfae278b9c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 298, 'output_tokens': 11, 'total_tokens': 309, 'input_token_details': {'cache_read': 0}})]


In [260]:
AIMessage.tool_calls

[{'name': 'get_random_number',
  'args': {'min_val': 1.0, 'max_val': 40.0},
  'id': '541faac0-9b92-4be4-a7f3-b7dfae278b9c',
  'type': 'tool_call'}]

In [261]:
from langchain_core.tools import Tool
for tools_calls in AIMessage.tool_calls:
  selected_tools={
      "get_current_date":get_current_date,
      "get_random_number":get_random_number,
      "get_current_time":get_current_time,
      "reverse_string":reverse_string,
      "calculate_area_of_rectangle":calculate_area_of_rectangle
  }[tools_calls["name"].lower()]
  print(selected_tools)
tools_msg=selected_tools.invoke(tools_calls)
print(tools_msg)
human_message.append(tools_msg)
print(human_message)


name='get_random_number' description='Returns a random integer between min_val and max_val (inclusive).' args_schema=<class 'langchain_core.utils.pydantic.get_random_number'> func=<function get_random_number at 0x7b4a1c1c71c0>
content='10' name='get_random_number' tool_call_id='541faac0-9b92-4be4-a7f3-b7dfae278b9c'
[HumanMessage(content='Give me the random number between 1 and 40 ', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_random_number', 'arguments': '{"min_val": 1.0, "max_val": 40.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-f7648f23-24d8-46ce-a67b-168a5e1ab651-0', tool_calls=[{'name': 'get_random_number', 'args': {'min_val': 1.0, 'max_val': 40.0}, 'id': '541faac0-9b92-4be4-a7f3-b7dfae278b9c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 298, 'output_tokens': 11, 'total_tokens': 309, 'input_token_detai

In [262]:
Actualresponce=llm.invoke(human_message)
print("Answer From LLM Using Tools : ",Actualresponce.content)

Answer From LLM Using Tools :  10
